<a href="https://colab.research.google.com/github/sagorbrur/my_colab_notebook/blob/master/Pytorch_Learning_Dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch, torchvision
torch.__version__

'1.3.1'

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
data = pd.read_table(url, header = None, sep = ",")
data.columns = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
data.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
data['class'] = data['class'].astype('category').cat.codes
data.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
X_data = data[["sepal_len", "sepal_wid", "petal_len", "petal_wid"]].values.astype('float32')
y_data = data['class'].values.astype('int32')
print(X_data.shape, y_data.shape)

(150, 4) (150,)


In [7]:
# Generating Tensor

X_tensor = torch.from_numpy(X_data)
y_tensor = torch.from_numpy(y_data)
print(X_tensor.type(), y_tensor.type())
print(X_tensor.size(), y_tensor.size())

torch.FloatTensor torch.IntTensor
torch.Size([150, 4]) torch.Size([150])


In [8]:
# assigning data type explicitly
X_tensor = torch.FloatTensor(X_data)
y_tensor = torch.LongTensor(y_data)

print(X_tensor.type(), y_tensor.type())
print(X_tensor.size(), y_tensor.size())

torch.FloatTensor torch.LongTensor
torch.Size([150, 4]) torch.Size([150])


In [9]:
# Logistic Regression model
model = torch.nn.Linear(X_data.shape[-1], len(set(y_data)))
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model

Linear(in_features=4, out_features=3, bias=True)

In [10]:
for epoch in range(100):
  outputs = model(X_tensor)
  loss = criterion(outputs, y_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+1)%10==0:
    print("Epoch: {} Loss: {:.5f}".format(epoch+1, loss.item()))

Epoch: 10 Loss: 1.75256
Epoch: 20 Loss: 1.66084
Epoch: 30 Loss: 1.57523
Epoch: 40 Loss: 1.49664
Epoch: 50 Loss: 1.42623
Epoch: 60 Loss: 1.36469
Epoch: 70 Loss: 1.31203
Epoch: 80 Loss: 1.26753
Epoch: 90 Loss: 1.22998
Epoch: 100 Loss: 1.19793


In [0]:
# Generating custom data with Dataloader
class IrisDataset(torch.utils.data.Dataset):
  def __init__(self):
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    data = pd.read_table(url, header = None, sep = ",")
    data.columns = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
    data["class"] = data["class"].astype("category").cat.codes
    
    self.X = data[["sepal_len", "sepal_wid", "petal_len", "petal_wid"]].values
    self.Y = data["class"].values[:, np.newaxis].astype(np.int32)

  def __getitem__(self, idx):
    # get item by index
    return self.X[idx], self.Y[idx]
  
  def __len__(self):
    # returns length of data
    return len(self.X)




In [14]:
irisDataset = IrisDataset()
print(type(irisDataset))
print(len(irisDataset))

<class '__main__.IrisDataset'>
150


In [15]:
dataLoader = torch.utils.data.DataLoader(irisDataset, batch_size=32, shuffle=True)
dataLoader

In [16]:
model = torch.nn.Linear(4, 3).double()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model

Linear(in_features=4, out_features=3, bias=True)

In [18]:
steps = len(dataLoader)

for epoch in range(100):
  for i, (x, y) in enumerate(dataLoader):
    outputs = model(x)
    loss = criterion(outputs, y.type(torch.LongTensor).view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1)%10==0:
    print("Epoch: {}, Loss: {:.5f}".format(epoch+1, loss.item()))



Epoch: 10, Loss: 3.10617
Epoch: 20, Loss: 2.30601
Epoch: 30, Loss: 2.27325
Epoch: 40, Loss: 1.71178
Epoch: 50, Loss: 1.20925
Epoch: 60, Loss: 1.08976
Epoch: 70, Loss: 0.91490
Epoch: 80, Loss: 0.85722
Epoch: 90, Loss: 0.86827
Epoch: 100, Loss: 0.82774


In [0]:
# Reference link:
#https://github.com/buomsoo-kim/PyTorch-learners-tutorial/blob/master/PyTorch%20Basics/pytorch-datasets-1.ipynb